In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,mls,mls
band,[3],3a
commitnumber,1013f91,a06b618
conc,-,-
dv,-,0.001
klin,-,0
molecule,{'h2o': 'atmpro'},h2o
ng_adju,-,"[0, 0]"
ng_refs,-,"[2, 6]"
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  0.000306                    1  0.000714
0.000750        2  0.000338                    2  0.000740
0.001052        3  0.000408                    3  0.000809
0.001476        4  0.000485                    4  0.000879
0.002070        5  0.000564                    5  0.000949
0.002904        6  0.000645                    6  0.001021
0.004074        7  0.000728                    7  0.001094
0.005714        8  0.000814                    8  0.001170
0.008015        9  0.000900                    9  0.001247
0.011243       10  0.000988                   10  0.001326
0.015771       11  0.001076                   11  0.001407
0.022122       12  0.001156                   12  0.001490
0.031031       13  0.001237                   13  0.001573
0.043528       14  0.001319                   14  0.001663
0.061057       15  0.001612                   15  0.001946
0.085645       16  0.002570                   16  0.002822
0.120136       17  0.004293                   17  0.004366
0.168516       18  0.006547                   18  0.006381
0.236378       19  0.009388                   19  0.008925
0.331549       20  0.012901                   20  0.012091
0.465100       21  0.017175                   21  0.016007
0.652400       22  0.022282                   22  0.020864
0.915100       23  0.026755                   23  0.025297
1.283650       24  0.026096                   24  0.025376
1.800600       25  0.021005                   25  0.021415
2.525700       26  0.015922                   26  0.016975
3.542800       27  0.011888                   27  0.013045
4.969550       28  0.008907                   28  0.009806
6.970850       29  0.006844                   29  0.007308
9.778100       30  0.005414                   30  0.005433
13.715850      31  0.004315                   31  0.003991
19.239350      32  0.003428                   32  0.002900
26.987250      33  0.002829                   33  0.002197
37.855300      34  0.002450                   34  0.001774
53.100050      35  0.002076                   35  0.001427
73.887500      36  0.001701                   36  0.001139
97.662500      37  0.001497                   37  0.001014
121.437500     38  0.001433                   38  0.000999
145.212500     39  0.001360                   39  0.000977
168.987500     40  0.001441                   40  0.001074
192.762500     41  0.002803                   41  0.002146
216.537500     42  0.009696                   42  0.007782
240.312500     43  0.023715                   43  0.020302
264.087500     44  0.043727                   44  0.038805
287.862500     45  0.066666                   45  0.059698
311.637500     46  0.079492                   46  0.073265
335.412500     47  0.091192                   47  0.086261
359.187500     48  0.105893                   48  0.099104
382.962500     49  0.122974                   49  0.111927
406.737500     50  0.142048                   50  0.127864
430.512500     51  0.162786                   51  0.149044
454.287500     52  0.181591                   52  0.170674
478.062500     53  0.200566                   53  0.192058
501.837500     54  0.221155                   54  0.213113
525.612500     55  0.243002                   55  0.234895
549.387500     56  0.265772                   56  0.257499
573.162500     57  0.289409                   57  0.282136
596.937500     58  0.326749                   58  0.322104
620.712500     59  0.362545                   59  0.362367
644.487500     60  0.392965                   60  0.398432
668.262500     61  0.417643                   61  0.428138
692.037500     62  0.435872                   62  0.449651
715.812500     63  0.447775                   63  0.464580
739.587500     64  0.453597                   64  0.474463
763.362500     65  0.453704                   65  0.480100
78

# Fluxes by Layer

CRD                              CLIRAD  \
                        flug          fldg      fnetg       flug   
pressure    level                                                  
0.000000    1     -29.471370  0.000000e+00 -29.471370 -29.569256   
0.000624    2     -29.471371  2.544421e-07 -29.471370 -29.569256   
0.000876    3     -29.471371  3.578625e-07 -29.471370 -29.569256   
0.001229    4     -29.471371  5.057844e-07 -29.471370 -29.569256   
0.001723    5     -29.471371  7.176776e-07 -29.471370 -29.569257   
0.002417    6     -29.471371  1.021134e-06 -29.471370 -29.569257   
0.003391    7     -29.471372  1.455738e-06 -29.471370 -29.569257   
0.004757    8     -29.471372  2.078138e-06 -29.471370 -29.569257   
0.006672    9     -29.471373  2.969378e-06 -29.471370 -29.569258   
0.009359    10    -29.471374  4.245551e-06 -29.471370 -29.569258   
0.013128    11    -29.471375  6.072794e-06 -29.471369 -29.569259   
0.018415    12    -29.471377  8.688333e-06 -29.471369 -29.569261   
0.025830    13    -29.471380  1.242354e-05 -29.471368 -29.569262   
0.036232    14    -29.471384  1.775887e-05 -29.471366 -29.569265   
0.050823    15    -29.471389  2.538101e-05 -29.471364 -29.569268   
0.071291    16    -29.471396  3.672006e-05 -29.471360 -29.569273   
0.100000    17    -29.471406  5.541796e-05 -29.471351 -29.569278   
0.140271    18    -29.471419  8.823484e-05 -29.471331 -29.569285   
0.196760    19    -29.471432  1.456008e-04 -29.471287 -29.569289   
0.275997    20    -29.471444  2.448911e-04 -29.471199 -29.569288   
0.387100    21    -29.471444  4.147570e-04 -29.471029 -29.569271   
0.543100    22    -29.471414  7.019209e-04 -29.470712 -29.569220   
0.761700    23    -29.471313  1.178362e-03 -29.470135 -29.569096   
1.068500    24    -29.471084  1.920968e-03 -29.469162 -29.568847   
1.498800    25    -29.470723  2.891462e-03 -29.467832 -29.568475   
2.102400    26    -29.470297  3.966982e-03 -29.466331 -29.568034   
2.949000    27    -29.469832  5.098360e-03 -29.464734 -29.567547   
4.136600    28    -29.469339  6.278173e-03 -29.463062 -29.567029   
5.802500    29    -29.468829  7.526031e-03 -29.461303 -29.566491   
8.139200    30    -29.468309  8.899444e-03 -29.459409 -29.565943   
11.417000   31    -29.467791  1.048451e-02 -29.457307 -29.565407   
16.014700   32    -29.467333  1.237540e-02 -29.454956 -29.564947   
22.464000   33    -29.467028  1.469063e-02 -29.452338 -29.564665   
31.510500   34    -29.466973  1.766612e-02 -29.449306 -29.564661   
44.200100   35    -29.467279  2.165503e-02 -29.445624 -29.565061   
62.000000   36    -29.468242  2.699556e-02 -29.441246 -29.566142   
85.775000   37    -29.470254  3.379793e-02 -29.436455 -29.568215   
109.550000  38    -29.472795  4.055539e-02 -29.432239 -29.570702   
133.325000  39    -29.475685  4.748216e-02 -29.428203 -29.573436   
157.100000  40    -29.478906  5.453246e-02 -29.424374 -29.576372   
180.875000  41    -29.482498  6.218342e-02 -29.420315 -29.579559   
204.650000  42    -29.487828  7.540762e-02 -29.412421 -29.584263   
228.425000  43    -29.500190  1.150761e-01 -29.385114 -29.595445   
252.200000  44    -29.523995  2.056696e-01 -29.318326 -29.617454   
275.975000  45    -29.564609  3.694335e-01 -29.195176 -29.655656   
299.750000  46    -29.628512  6.210901e-01 -29.007423 -29.716506   
323.525000  47    -29.709900  9.263533e-01 -28.783547 -29.794278   
347.300000  48    -29.806195  1.279473e+00 -28.526721 -29.886505   
371.075000  49    -29.917983  1.689490e+00 -28.228492 -29.993855   
394.850000  50    -30.045627  2.163468e+00 -27.882160 -30.116669   
418.625000  51    -30.189239  2.707132e+00 -27.482108 -30.255173   
442.400000  52    -30.348964  3.325313e+00 -27.023650 -30.409812   
466.175000  53    -30.521851  4.009617e+00 -26.512233 -30.577706   
489.950000  54    -30.706450  4.759075e+00 -25.947375 -30.757226   
513.725000  55    -30.902648  5.578116e+00 -25.324532 -30.948180   
537.500000  56    -31.110156  6.469995e+00 -24.640161 -31.150772   
561.275000  5

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')